In [27]:
import pandas as pd
import numpy as np
import os, random, math, glob
from IPython.display import Image as IM
from IPython.display import clear_output
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import lightgbm  as lg
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.neural_network import MLPClassifier

from sklearn.linear_model import LinearRegression
from math import sqrt

#Transformation
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]

plt.rcParams['figure.figsize'] = [16, 10]

In [28]:
df = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')
df["Avg_ind_house"]=df["total_households"]/df["total_individuals"]
df_test["Avg_ind_house"]=df_test["total_households"]/df_test["total_individuals"]



In [29]:
X = df[[ "Avg_ind_house", 'dw_00',
       'dw_01', 'dw_02', 'dw_03', 'dw_04', 'dw_05', 'dw_06', 'dw_07', 'dw_08',
       'dw_09', 'dw_10', 'dw_11', 'dw_12', 'dw_13', 'psa_00', 'psa_01',
       'psa_02', 'psa_03', 'psa_04', 'stv_00', 'stv_01', 'car_00', 'car_01',
       'lln_00', 'lln_01', 'lan_00', 'lan_01', 'lan_02', 'lan_03', 'lan_04',
       'lan_05', 'lan_06', 'lan_07', 'lan_08', 'lan_09', 'lan_10', 'lan_11',
       'lan_12', 'lan_13', 'lan_14', 'pg_00', 'pg_01', 'pg_02', 'pg_03',
       'pg_04', 'lgt_00', 'pw_00', 'pw_01', 'pw_02', 'pw_03', 'pw_04', 'pw_05',
       'pw_06', 'pw_07', 'pw_08',  'lat', 'lon', 'NL']]

Xtest = df_test[[ "Avg_ind_house",'dw_00',
       'dw_01', 'dw_02', 'dw_03', 'dw_04', 'dw_05', 'dw_06', 'dw_07', 'dw_08',
       'dw_09', 'dw_10', 'dw_11', 'dw_12', 'dw_13', 'psa_00', 'psa_01',
       'psa_02', 'psa_03', 'psa_04', 'stv_00', 'stv_01', 'car_00', 'car_01',
       'lln_00', 'lln_01', 'lan_00', 'lan_01', 'lan_02', 'lan_03', 'lan_04',
       'lan_05', 'lan_06', 'lan_07', 'lan_08', 'lan_09', 'lan_10', 'lan_11',
       'lan_12', 'lan_13', 'lan_14', 'pg_00', 'pg_01', 'pg_02', 'pg_03',
       'pg_04', 'lgt_00', 'pw_00', 'pw_01', 'pw_02', 'pw_03', 'pw_04', 'pw_05',
       'pw_06', 'pw_07', 'pw_08',  'lat', 'lon', 'NL']]
y = df["target"]

In [23]:
print(X.shape)
print(Xtest.shape)

(2822, 59)
(1013, 59)


In [21]:
X = df[['lln_00', 'lan_06', 'psa_04', 'lat', 'lan_03', 'lan_07', 'psa_01',
       'lan_09', 'dw_01', 'lan_05', 'NL', 'lln_01', 'pg_00', 'lan_11', "lon",
       'lan_10', 'car_00', 'car_01', 'pw_00', 'psa_00']]

y= df["target"]

Xtest= df_test[['lln_00', 'lan_06', 'psa_04', 'lat', 'lan_03', 'lan_07', 'psa_01',
       'lan_09', 'dw_01', 'lan_05', 'NL', 'lln_01', 'pg_00', 'lan_11', "lon",
       'lan_10', 'car_00', 'car_01', 'pw_00', 'psa_00']]

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# ALL MODELS 

In [54]:
scoring = 'neg_mean_squared_error'

# Define models 
names = [" Random Forest", "AdaBoost","XGBoost", 
          "K Nearest Neighbours"]

regressors = [
     RandomForestRegressor(n_estimators=466, min_samples_split=5, max_features= 'sqrt',
                           min_samples_leaf=1,max_depth= 90, bootstrap= False),
     AdaBoostRegressor(n_estimators= 800, loss= 'square', learning_rate= 0.83,
                             random_state=None),
     XGBRegressor(n_estimators=1000,learning_rate=0.05, min_child_weight=2,max_depth=4,
                         eta =0.6, reg_lambda = 2),
     KNeighborsRegressor(n_neighbors=3, p=2, n_jobs=10)]
seed = 1


In [102]:
models = zip(names, regressors)
results = []
names = []#
for name, model in models:
    kfold = KFold(n_splits=10, random_state=seed)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, predictions))
    #print("The RMSE for {} is {}".format(name, rmse) )

/home/aims/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/home/aims/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/home/aims/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/home/aims/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future versi

[23:50:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/aims/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[23:50:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/aims/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

/home/aims/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[23:51:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/aims/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[23:51:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/aims/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


# PARAMETER TUNNING FOR ADABOOST

In [56]:
param_dist = {
 'n_estimators':np.arange(100,2000, 50),
 'learning_rate' : np.arange(0.01,1, 0.01),
    'colsample_bytree': np.arange(0.1, 1, 0.1),
    'max_depth': np.arange(10, 40, 5),
    'reg_alpha': np.arange(1.1, 2, 0.1),
    'reg_lambda': np.arange(1.1, 2, 0.1),
    'subsample':np.arange(0.7, 1, 0.1)
 }
    
pre_gs_inst = RandomizedSearchCV(XGBRegressor(),
 param_distributions = param_dist,
 cv=2,
 n_iter = 40,
 n_jobs=-1)

pre_gs_inst.fit(X_train, y_train)
pre_gs_inst.best_params_


/home/aims/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[22:19:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_state=0, reg_alpha=...
        650,  700,  750,  800,  850,  900,  950, 1000, 1050, 1100, 1150,
       1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700,
       1750, 1800, 1850, 1900, 1950]),
   

# Parameter tunning for Xgboost

In [50]:
param_dist = {
 'n_estimators':np.arange(50,1000, 50),
 'learning_rate' : np.arange(0.01,1, 0.01),
 'loss' : ['neg_mean_squared_error', 'square']
 }

pre_gs_inst = RandomizedSearchCV(AdaBoostRegressor(),
 param_distributions = param_dist,
 cv=2,
 n_iter = 40,
 n_jobs=-1)

pre_gs_inst.fit(X_train, y_train)
pre_gs_inst.best_params_


RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=AdaBoostRegressor(base_estimator=None,
                                               learning_rate=1.0, loss='linear',
                                               n_estimators=50,
                                               random_state=None),
                   iid='deprecated', n_iter=40, n_jobs=-1,
                   param_distributions={'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0...
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]),
                                        'loss': ['neg_mean_squared_error',
                                                 'square'],
                                        'n_estimators': array([ 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 55

# ADABOOST

In [46]:
model_ada = AdaBoostRegressor(n_estimators= 4200, loss= 'square', learning_rate= 0.05,
                             random_state=None)

model_ada.fit(X_train, y_train)
predictions_ada = model_ada.predict(X_test)
sqrt(mean_squared_error(y_test,predictions_ada ))

3.844647835909405

# RANDOM FOREST

In [47]:
rf = RandomForestRegressor(n_estimators=466, min_samples_split=5, max_features= 'sqrt',
                           min_samples_leaf=1,max_depth= 90, bootstrap= False)
rf.fit(X_train, y_train)
predictions= rf.predict(X_test)
print("RMSE:", sqrt(mean_squared_error(y_test, predictions)))

RMSE: 3.271901846771979


## XGBOOST REGRESSOR MODEL

In [48]:
model_xgb = XGBRegressor(n_estimators=1300,learning_rate=0.05, min_child_weight=2,max_depth=4,
                         eta =0.4, reg_lambda = 2.0)

model_xgb.fit(X_train, y_train)
sqrt(mean_squared_error(y_test, model_xgb.predict(X_test)))

[00:39:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/aims/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


3.0831350066173577

## Combined xgboost, adaboost and random forest

In [49]:
avg_pre = (model_xgb.predict(X_test)*0.6 + rf.predict(X_test)*0.3+ model_ada.predict(X_test)*0.1)

sqrt(mean_squared_error(y_test, avg_pre))

3.079592979635759

## Save in csv 

In [50]:
def create_submission(data, fname, cols, models=[]):
    score = 0
    for model in models:
        score += model.predict(data[cols])
    data['target'] = (score / len(models))
    submissions = data[['ward', 'target']]
    submissions.to_csv(fname, index=False)
    return submissions

In [51]:
top_cols_20 = ["Avg_ind_house",  'dw_00',
       'dw_01', 'dw_02', 'dw_03', 'dw_04', 'dw_05', 'dw_06', 'dw_07', 'dw_08',
       'dw_09', 'dw_10', 'dw_11', 'dw_12', 'dw_13', 'psa_00', 'psa_01',
       'psa_02', 'psa_03', 'psa_04', 'stv_00', 'stv_01', 'car_00', 'car_01',
       'lln_00', 'lln_01', 'lan_00', 'lan_01', 'lan_02', 'lan_03', 'lan_04',
       'lan_05', 'lan_06', 'lan_07', 'lan_08', 'lan_09', 'lan_10', 'lan_11',
       'lan_12', 'lan_13', 'lan_14', 'pg_00', 'pg_01', 'pg_02', 'pg_03',
       'pg_04', 'lgt_00', 'pw_00', 'pw_01', 'pw_02', 'pw_03', 'pw_04', 'pw_05',
       'pw_06', 'pw_07', 'pw_08',  'lat', 'lon', 'NL']
len(top_cols_20)

59

In [52]:
create_submission(df_test, 'overfittin_ada.csv', cols=top_cols_20, models=[rf,model_ada, model_xgb])

,ward,target
0,21001001: Ward 1,16.743114
1,21001002: Ward 2,13.494306
2,21001003: Ward 3,21.387680
3,21001004: Ward 4,15.799457
4,21001005: Ward 5,20.528883
5,21001006: Ward 6,23.528814
6,21001007: Ward 7,13.401826
7,21002001: Ward 1,16.736699
8,21002002: Ward 2,16.228856
9,21002003: Ward 3,21.381478
